In [0]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.5'

# 4.1 - Redes Neurais Convolutivas - Introdução

No exemplo inicial, criamos uma rede neural para reconhecer dígitos escritos com letra cursiva. Como base de dados utilizamos o dataset MNIST. A rede utilizou camadas com conexão densa. Uma conexão do tipo densa é aquela onde todos os neurons de um nível estão conectados a todos os neurons do nível seguinte. Com esta estrutura obteve-se uma precisão no conjunto de teste de ~ 97.8%. 

Neste exemplo vamos utilizar um outro tipo de arquitetura de rede, denominada convolutiva. Nele ao invés de utilizar camadas do tipo `layers.Dense` utilizamos camadas do tipo `layers.Conv2D` e `layers.MaxPooling2D`. 

Este tipo de rede recebe tensores de entrada no formato `(image_height, image_width, image_channel)`, além do número de tensores enviados à rede por "batch" de treino. 

Lembrando: o dataset MNIST contém imagens em tons de cinza (um único canal) em pixels dispostos em um quadrado 28x28. O formato do tensor de entrada do primeiro nível será dado por `input_shape=(28, 28, 1)`.

In [1]:
from keras import layers
from keras import models

# Define o tipo de rede, neste caso sequencial
model = models.Sequential()

# Define uma camada convolutiva com um tensor 3D de entrada no formato (28,28,1). 
#  Sendo assim a entrada é formada por imagens de 28x28 pixels com um canal de 
#   cor (ton de cinza). 
#  Neste nível teremos 32 kernels de convolução. Cada kernel terá formato (3,3,1).
#  Supondo Padding=0 (sem beiradas externas nas entradas 28x28) e um avanço do
#  do kernel (stride) igual a 1, as dimensões da saida serão 28 - 3 + 1 = 26.
#  Como temos 32 kernels, a saida será um tensor 3D no formato (26, 26, 32), isto é
#  32 canais cada um com uma imagem de 26x26 pixels.    
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

# Define uma camada de Pooling2D, a qual utiliza a função Max. Esta camada 
#  aplica a função Max em grupos de 2x2 pixels ao longo de cada imagem 26x26 
#  gerando uma nova imagem 13x13
model.add(layers.MaxPooling2D((2, 2)))

# A camada a seguir transforma a entrada de 32 canais, cada um com uma imagem
#  13x13 em uma saida com 64 canais, cada um com uma imagem 11x11. 
#  Neste caso supondo cada kernel no formato (3,3), Padding=0 e Stride=1 teremos
#  a dimensão de saida igual a: 13 - 3 + 1 = 11.
#  Como dimensionamos 64 kernels teremos 64 imagens 11x11.
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Esta segunda camada de pooling executa em cada um dos 64 canais de entrada
#  (cada um deles com uma imagem 11x11) em uma nova imagem 5x5, através da
#  aplicação da função Max em grupos de 2x2 pixels.
model.add(layers.MaxPooling2D((2, 2)))

# Por último temos uma camada convolutiva com 64 canais, cada um com uma imagem
# de 5x5 pixels. O formato de saida deste nível convolutivo é um tensor 3D 
# do tipo (3,3,64).
# Isto é confirmado supondo Padding = 0, Stride = 1 e Kernels no formato (3,3),
# logo a dimensão de saida será 5 - 3 + 1 = 3.
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

Using TensorFlow backend.


A célula a seguir apresenta um resumo da arquitetura de rede utilizada até o momento.

In [2]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


Pelo resumo apresentado acima, pode-se ver que a saida das camadas `Conv2D` and `MaxPooling2D` é sempre um tensor 3D com formato `(height, width, channels)`. 

Além disso, a medida que avançamos nas camadas da rede, o número de dimensões relativas à altura e à largura de cada uma das imagens (o número de pixels que a representa) tende a diminuir. 

A próxima etapa consiste em passar o último tensor (neste exemplo com formato `(3, 3, 64)` para uma rede de classificação com camadas do tipo `Dense`. 

Este tipo de camada processa vetores (tensores 1D) e o tensor de saida da última camada convolutiva é do tipo 3D. Sendo assim, é necessário "alisar" (flatten em inglês) os tensores 3D em vetores (tensores 1D) para que eles possam ser enviados a primeira camada do tipo `Dense`.

Este processo é feito por uma cada específica do tipo `Flatten`

In [0]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

Lembrando que o nosso problema requer a escolha de um número entre dez possibilidades (classificação em 10 níveis), são utilizados 10 neurons na camada de saida e a função de ativação escolhida é do tipo `softmax`.

In [4]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)               

Como pode ser observado pelo resumo da rede, as saidas no formato `(3, 3, 64)` do último nível convolutivo são "alisadas" (transformadas) em vetores no formato 3*3*64 = 576 `(576,)` antes de serem passadas para as camadas do tipo `Dense`.

Vamos agora proceder com preparação dos dados disponíveis no dataset MNIST, dividindo os mesmos em grupos de treino e teste, reformatando os mesmos de acordo com o formato de entrada dos tensores do primeiro nível da nossa rede neural e normalizando os valores originais de entrada (de 0 a 255) para a faixa (0 a 1). 

In [5]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Observe que reformatamos train_images de (60000,28,28) para (60000,28,28,1)
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

# Idem para o grupo de teste de (10000,28,28) para (10000,28,28,1)
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

# Por último procedemos com a "dumificação" (vetorização) dos rótulos de saida
#  passando de uma lista com 60000 valores de 0 a 9 para uma matriz (60000,10) 
#  composta apenas de 0s e 1s (no caso do conjunto de treino)
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 1s 0us/step


Em seguida compilamos e treinamos o modelo. Os 60.000 tensores 3D de entrada, cada um no formato (28,28,1), serão lidos em grupos (batches) de 64 tensores. 

O conjunto total dos 60.000 tensores será percorrido cinco vezes (`epochs=5`).

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/5





60000/60000 [==============================] - 19s 311us/step - loss: 0.1783 - acc: 0.9439
Epoch 2/5
60000/60000 [==============================] - 5s 79us/step - loss: 0.0472 - acc: 0.9849
Epoch 3/5
60000/60000 [==============================] - 5s 80us/step - loss: 0.0324 - acc: 0.9894
Epoch 4/5
60000/60000 [==============================] - 5s 79us/step - loss: 0.0241 - acc: 0.9925
Epoch 5/5
60000/60000 [==============================] - 5s 79us/step - loss: 0.0190 - acc: 0.9942


Por último vamos avaliar a precisão do modelo no conjunto de teste:

In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)

10000/10000 [==============================] - 1s 61us/step


In [8]:
test_acc

0.9893

Observe que enquanto o modelo do exemplo anterior, com camadas do tipo `Dense` obteve uma precisão de ~ 97.8%, o modelo convolutivo aqui apresentado obteve uma precisão no teste de ~ 99.2% (os valores exatos podem variar).

A pergunta que será respondida nos próximos exercícios é: o que ocorre em uma camada do tipo convolutivo que permite tal melhoria no erro?